In [ ]:
!pip install --upgrade evaluate jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 50.8 MB/s eta 0:00:00


In [ ]:
from evaluate import load

In [ ]:
wer_metric = load('wer')

In [ ]:
reference = "the cat sat on the mat"
prediction = "the cat sit on the"

In [ ]:
wer = wer_metric.compute(references=[reference], predictions=[prediction])

In [ ]:
wer

0.3333333333333333

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

In [ ]:
prediction = " He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similarly is drawn from eating and its results occur most readily to the mind."

In [ ]:
normalizer = BasicTextNormalizer()
normalized_prediction = normalizer(prediction)
normalized_prediction

' he tells us that at this festive season of the year with christmas and roast beef looming before us similarly is drawn from eating and its results occur most readily to the mind '

In [ ]:
reference = "HE TELLS US THAT AT THIS FESTIVE SEASON OF THE YEAR WITH CHRISTMAS AND ROAST BEEF LOOMING BEFORE US SIMILES DRAWN FROM EATING AND ITS RESULTS OCCUR MOST READILY TO THE MIND"
normalized_referece = normalizer(reference)

wer = wer_metric.compute(
    references=[normalized_referece], predictions=[normalized_prediction]
)
wer

0.0625

In [ ]:
from transformers import pipeline
import torch

if torch.cuda.is_available():
  device = 'cuda:0'
  dtype  = torch.float16
else:
  device = 'cpu'
  dtype = torch.float32

pipe = pipeline(
    'automatic-speech-recognition',
    model='openai/whisper-small',
    torch_dtype=dtype,
    device=device
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
common_voice_test = load_dataset(
    'mozilla-foundation/common_voice_13_0',
    'dv',
    split='test'
)


In [ ]:
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

all_predictions = []

for prediction in tqdm(
        pipe(
            KeyDataset(common_voice_test, "audio"),
            max_new_tokens=128,
            d={"task":
                             "transcribe",
                             "language":'Sinhala'
                             },
            batch_size=32,
        ),
        total=len(common_voice_test),
    ):
    all_predictions.append(prediction["text"])

In [ ]:
from evaluate import load

In [ ]:
wer_metric = load('wer')
wer_ortho = 100*wer_metric.compute(references=common_voice_test['sentence'], predictions=all_predictions)
wer_ortho

171.85040740998676

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

In [ ]:
normalizer = BasicTextNormalizer()

all_pred_norm = [normalizer(ele) for ele in all_predictions]
all_ref_norm = [normalizer(ele) for ele in common_voice_test['sentence']]

all_pred_norm = [ele for ele in all_pred_norm if len(ele) > 0]
all_ref_norm = [ele for ele in all_ref_norm if len(ele)>0]

wer_ortho = 100*wer_metric.compute(references=all_ref_norm, predictions=all_pred_norm)
wer_ortho

128.02621970302883